# Usage Example

In this notebook we will show the most basic usage of **TGAN** in order to generate samples from a
given dataset.

### 1. Load the data

The first step is to load the data wich we will use to fit TGAN. In order to do so, we will first
import the function `tgan.data.load_data` and call it with the name the dataset that we want to load.

In this case, we will load the `census` dataset, which we will use during the subsequent steps, and obtain two objects:

1. `data` will contain a `pandas.DataFrame` with the table of data from the `census` dataset ready to be used to fit the model.

2. `continous_columns` will contain a `list` with the indices of continuous columns.

In [3]:
import sys
sys.path.append('..')

In [4]:
from tgan.data import load_demo_data

data, continuous_columns = load_demo_data('census')

data.head(3).T

,0,1,2
0,73,58,18
1,Not in universe,Self-employed-not incorporated,Not in universe
2,0,4,0
3,0,34,0
4,High school graduate,Some college but no degree,10th grade
5,0,0,0
6,Not in universe,Not in universe,High school
7,Widowed,Divorced,Never married
8,Not in universe or children,Construction,Not in universe or children
9,Not in universe,Precision production craft & repair,Not in universe


In [5]:
continuous_columns

[0, 5, 16, 17, 18, 29, 38]

### 2. Create a TGAN instance

The next step is to import TGAN and create an instance of the model.

To do so, we need to import the `tgan.model.TGANModel` class and call it.

This will create a TGAN instance with the default parameters.


In [6]:
from tgan.model import TGANModel

tgan = TGANModel(continuous_columns)

### 3. Fit the model

The third step is to pass the data that we have loaded previously to the `TGANModel.fit` method to
start the fitting.

This process will not return anything, however, the progress of the fitting will be printed into screen.

**NOTE** Depending on the performance of the system you are running, and the parameters selected
for the model, this step can take up to a few hours.


In [7]:
tgan.fit(data)

AttributeError: 'InputDesc' object has no attribute 'graph'

### 4. Sample new data

After the model has been fit, we are ready to generate new samples by calling the `TGANModel.sample`
method passing it the desired amount of samples.

The returned object, `samples`, is a `pandas.DataFrame` containing a table of synthetic data with
the same format as the input data and 1000 rows as we requested.

In [ ]:
num_samples = 1000

samples = tgan.sample(num_samples)

samples.head(3)

### 5. Save and Load a model

In the steps above we saw that the fitting process is slow, so we probably would like to avoid having to fit every we want to generate samples. Instead we can fit a model once, save it, and load it every time we want to sample new data.

If we have a fitted model, we can save it by calling the `TGANModel.save` method, that only takes
as argument the path to store the model into. Similarly, the `TGANModel.load` allows to load a model stored on disk by passing as argument a path where the model is stored.

At this point we could use this model instance to generate more samples.

In [ ]:
model_path = 'demo/my_model'

tgan.save(model_path)

In [ ]:
new_tgan = TGANModel.load(model_path)

In [ ]:
new_samples = new_tgan.sample(num_samples)

new_samples.head(3)

## Loading custom datasets

In the previous steps we used some demonstration data but we did not show how to load your own dataset.

In order to do so you can use `pandas.read_csv` by passing it the path to the CSV file that you want to load.

Additionally, you will need to create 0-indexed list of columns indices to be considered continuous.

For example, if we want to load a local CSV file, `path/to/my.csv`, that has as continuous columns their first 4 columns, that is, indices `[0,1,2,3]`, we would do it like this:

In [ ]:
import pandas as pd

data = pd.read_csv('data/census.csv')

continuous_columns = [0,1,2,3]

## Model Parameters

If you want to change the default behavior of TGANModel, such as as different `batch_size` or
`num_epochs`, you can do so by passing different arguments when creating the instance. Have b

### Model general behavior

* continous_columns (`list[int]`, required): List of columns to be considered continuous.
* output (`str`, default=`output`): Path to store the model and its artifacts.
* gpu (`list[str]`, default=`[]`): Comma separated list of GPU(s) to use.

### Neural network definition and fitting

* max_epoch (`int`, default=`100`): Number of epochs to use during training.
* steps_per_epoch (`int`, default=`10000`): Number of steps to run on each epoch.
* save_checkpoints(`bool`, default=True): Whether or not to store checkpoints of the model after each training epoch.
* restore_session(`bool`, default=True): Whether or not continue training from the last checkpoint.
* batch_size (`int`, default=`200`): Size of the batch to feed the model at each step.
* z_dim (`int`, default=`100`): Number of dimensions in the noise input for the generator.
* noise (`float`, default=`0.2`): Upper bound to the gaussian noise added to categorical columns.
* l2norm (`float`, default=`0.00001`): L2 reguralization coefficient when computing losses.
* learning_rate (`float`, default=`0.001`): Learning rate for the optimizer.
* num_gen_rnn (`int`, default=`400`):
* num_gen_feature (`int`, default=`100`): Number of features of in the generator.
* num_dis_layers (`int`, default=`2`):
* num_dis_hidden (`int`, default=`200`):
* optimizer (`str`, default=`AdamOptimizer`): Name of the optimizer to use during `fit`, possible
  values are: [`GradientDescentOptimizer`, `AdamOptimizer`, `AdadeltaOptimizer`].

If we wanted to create an identical instance to the one created on step 2, but passing the arguments in a explicit way we will do something like this:

In [ ]:
tgan = TGANModel(
    continuous_columns,
    output='output',
    gpu=None,
    max_epoch=5,
    steps_per_epoch=10000,
    save_checkpoints=True,
    restore_session=True,
    batch_size=200,
    z_dim=200,
    noise=0.2,
    l2norm=0.00001,
    learning_rate=0.001,
    num_gen_rnn=100,
    num_gen_feature=100,
    num_dis_layers=1,
    num_dis_hidden=100,
    optimizer='AdamOptimizer'
)

## Command-line interface

We include a command-line interface that allows users to access TGAN functionality. Currently only one action is supported.

### Random hyperparameter search

#### Input

To run random searchs for the best model hyperparameters for a given dataset, we will need:

* A dataset, in a csv file, without any missing value, only columns of type `bool`, `str`, `int` or
  `float` and only one type for column, as specified in [Data Format Input](#data-format-input).

* A JSON file containing the configuration for the search. This configuration shall contain:

  * `name`: Name of the experiment. A folder with this name will be created.
  * `num_random_search`: Number of iterations in hyper parameter search.
  * `train_csv`: Path to the csv file containing the dataset.
  * `continuous_cols`: List of column indices, starting at 0, to be considered continuous.
  * `epoch`: Number of epoches to train the model.
  * `steps_per_epoch`: Number of optimization steps in each epoch.
  * `sample_rows`: Number of rows to sample when evaluating the model.

You can see an example of such a json file in [examples/config.json](examples/config.json), which you
can download and use as a template.

#### Execution

Once we have prepared everything we can launch the random hyperparameter search with this command:

``` bash
tgan experiments config.json results.json
```

Where the first argument, `config.json`,  is the path to your configuration JSON, and the second,
`results.json`, is the path to store the summary of the execution.

This will run the random search, wich basically consist of the folling steps:

1. We fetch and split our data between test and train.
2. We randomly select the hyperparameters to test.
3. Then, for each hyperparameter combination, we train a TGAN model using the real training data T
   and generate a synthetic training dataset Tsynth.
4. We then train machine learning models on both the real and synthetic datasets.
5. We use these trained models on real test data and see how well they perform.

#### Output

One the experiment has finished, the following can be found:

* A JSON file, in the example above called `results.json`, containing a summary of the experiments.
  This JSON will contain a key for each experiment `name`, and on it, an array of length
  `num_random_search`, with the selected parameters and its evaluation score. For a configuration
  like the example, the summary will look like this:

``` python
{
    'census': [
        {
            "steps_per_epoch" : 10000,
            "num_gen_feature" : 300,
            "num_dis_hidden" : 300,
            "batch_size" : 100,
            "num_gen_rnn" : 400,
            "score" : 0.937802280415988,
            "max_epoch" : 5,
            "num_dis_layers" : 4,
            "learning_rate" : 0.0002,
            "z_dim" : 100,
            "noise" : 0.2
        },
        ... # 9 more nodes
    ]
}
```

* A set of folders, each one names after the `name` specified in the JSON configuration, contained
in the `experiments` folder. In each folder, sampled data and the models can be found. For a configuration
like the example, this will look like this:

```
experiments/
  census/
    data/       # Sampled data with each of the models in the random search.
    model_0/
      logs/     # Training logs
      model/    # Tensorflow model checkpoints
    model_1/    # 9 more folders, one for each model in the random search
    ...
```

## Citation

If you use TGAN, please cite the following work:

> Lei Xu, Kalyan Veeramachaneni. 2018. Synthesizing Tabular Data using Generative Adversarial Networks.

```LaTeX
@article{xu2018synthesizing,
  title={Synthesizing Tabular Data using Generative Adversarial Networks},
  author={Xu, Lei and Veeramachaneni, Kalyan},
  journal={arXiv preprint arXiv:1811.11264},
  year={2018}
}
```
You can find the original paper [here](https://arxiv.org/pdf/1811.11264.pdf)